In [ ]:
import pandas as pd
from os import path, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [ ]:
input_dir = path.join('dados')

In [ ]:
filename = path.join(input_dir, 'distritos_original.csv')
df_distritos = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_distritos

In [ ]:
filename = path.join(input_dir, 'subprefeituras_original.csv')
df_subs = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_subs

In [ ]:
filename = path.join(input_dir, 'censo_demografia_original.csv')
df_pop = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_pop

# Transformação dos dados

## Unindo as tabelas de distritos e subprefeituras

Vemos que as subprefeituras possuem duas colunas diferentes de id: `cd_identificador_subprefeitura` e `cd_subprefeitura`. Como as os valores nas duas colunas não coincidem, vamos avaliar qual delas é a mais adequada para fazermos a junção com os dados dos distritos.

In [ ]:
df_subs.sort_values('cd_identificador_subprefeitura').head(5)

In [ ]:
df_distritos.sort_values('cd_identificador_subprefeitura').head(5)

Vemos que ocorrem duas coisas: a coluna `cd_identificador_subprefeitura` da tabela de distritos coincide com o valor numérico da coluna `cd_subprefeitura`, mas a coluna `cd_subprefeitura` está em formato de texto, iniciando com zeros à esquerda. Portanto, vamos adicionar à tabela de distritos uma nova coluna `cd_subprefeitura`, convertendo o valor da coluna `cd_identificador_subprefeitura` para texto e preenchendo com zeros à esquerda para que a string tenha 2 caracteres.

In [ ]:
df_distritos.insert(
    3,
    'cd_subprefeitura',
    df_distritos['cd_identificador_subprefeitura'].astype(str).str.zfill(2)
)
df_distritos.head()

Agora, vamos unir as tabelas de distritos e subprefeituras, utilizando a coluna `cd_subprefeitura` como chave de junção.

In [ ]:
df_dist_sub = (
    df_distritos.drop(columns=['cd_identificador_subprefeitura'])
    .merge(
        df_subs.drop(columns=['cd_identificador_subprefeitura']),
        on='cd_subprefeitura',
        how='left'
    )
)

df_dist_sub.head()

Investigando o resultado no data wrangler, vemos que a junção ocorreu corretamente, com todos os distritos associados à sua respectiva subprefeitura.